### 1. Exploración del conjunto de datos:

In [7]:
import pandas as pd

# Cargar el dataset omitiendo la columna "Unnamed: 0"
df = pd.read_csv('Spotify_Youtube.csv', index_col=0)

# Visualizar todas las columnas del dataframe
pd.set_option('display.max_columns', None)

# Primeras 5 filas del dataframe
print("\nPrimeras 5 filas del dataframe:\n")
print(df.head())

# Últimas 5 filas del dataframe
print("\n\nÚltimas 5 filas del dataframe:\n")
print(df.tail())

# 10 filas aleatorias del dataframe
print("\n\n10 filas aleatorias del dataframe:\n")
print(df.sample(10))

# Número de filas y columnas del dataframe
print("\n\nNúmero de filas y columnas del dataframe:\n")
print(df.shape)

# Tipos de datos de cada columna
print("\n\nTipos de datos de cada columna:\n")
print(df.dtypes)

# Número de valores nulos por columna
print("\n\nNúmero de valores nulos por columna:\n")
print(df.isnull().sum())

# Verificar filas duplicadas
print("\n\nNúmero de filas duplicadas:\n")
print(df.duplicated().sum())

# Estadísticas para las columnas numéricas
print("\n\nEstadísticas descriptivas para las columnas numéricas:\n")
print(df.describe())

# Estadísticas para las columnas categóricas
print("\n\nEstadísticas descriptivas para las columnas categóricas:\n")
print(df.describe(include=['object', 'bool']))



Primeras 5 filas del dataframe:

     Artist                                        Url_spotify  \
0  Gorillaz  https://open.spotify.com/artist/3AA28KZvwAUcZu...   
1  Gorillaz  https://open.spotify.com/artist/3AA28KZvwAUcZu...   
2  Gorillaz  https://open.spotify.com/artist/3AA28KZvwAUcZu...   
3  Gorillaz  https://open.spotify.com/artist/3AA28KZvwAUcZu...   
4  Gorillaz  https://open.spotify.com/artist/3AA28KZvwAUcZu...   

                                           Track  \
0                                 Feel Good Inc.   
1                                Rhinestone Eyes   
2  New Gold (feat. Tame Impala and Bootie Brown)   
3                             On Melancholy Hill   
4                                 Clint Eastwood   

                                           Album Album_type  \
0                                     Demon Days      album   
1                                  Plastic Beach      album   
2  New Gold (feat. Tame Impala and Bootie Brown)     single   
3   

### 2. Preparación de los datos:

In [15]:
import pandas as pd

# Cargar el dataset omitiendo la columna "Unnamed: 0"
df = pd.read_csv('Spotify_Youtube.csv', index_col=0)


# 1. Convertir los nombres de las columnas a minúsculas
df.columns = df.columns.str.lower()


# 2. Crear una función para reemplazar las comas por puntos
def replace_commas_to_float(df, columns):
 
    for col in columns:
        if df[col].dtype == 'object':  # Verifica que la columna sea de tipo objeto (texto)
            df[col] = df[col].str.replace(',', '.')
        df[col] = pd.to_numeric(df[col], errors='coerce')  # Convierte a float, valores no convertibles a NaN
    return df

# Especificar las columnas que necesitan corrección
columns_to_correct = ['danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness']

# Aplicar la función
df_combined = replace_commas_to_float(df, columns_to_correct)

# Verificar los tipos de datos
print("\n\nVerificamos los tipos de datos:\n")
print(df[columns_to_correct].dtypes)


# 3. Crear una función para convertir a minúsculas los títulos y nombres de álbumes
def to_lowercase(df, columns):
    for col in columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.lower()

# Columnas a convertir
columns_to_lowercase = ['track', 'title', 'album']

# Aplicar la función
to_lowercase(df, columns_to_lowercase)

# Verificar los cambios
print("\n\nConvertimos a minúsculas los títulos y nombres de álbumes:\n")
print(df[columns_to_lowercase].head())


# 4. Crear una función para reemplazar el símbolo $ por s
def replace_dollar_sign(df, column):
    # Asegurarse de que la columna existe en el DataFrame
    if column in df.columns:
        df[column] = df[column].str.replace('$', 's', regex=False)

# Aplicar la función a la columna 'artist'
replace_dollar_sign(df, 'artist')

# Verificar los cambios
print("\n\nReemplazamos el símbolo $ por s:\n")
print(df[['artist']].sample(5))

# Guardar el DataFrame transformado
df.to_csv('Spotify_Youtube.csv', index=False)



Verificamos los tipos de datos:

danceability        float64
energy              float64
key                 float64
loudness            float64
speechiness         float64
acousticness        float64
instrumentalness    float64
dtype: object


Convertimos a minúsculas los títulos y nombres de álbumes:

                                           track  \
0                                 feel good inc.   
1                                rhinestone eyes   
2  new gold (feat. tame impala and bootie brown)   
3                             on melancholy hill   
4                                 clint eastwood   

                                               title  \
0         gorillaz - feel good inc. (official video)   
1  gorillaz - rhinestone eyes [storyboard film] (...   
2  gorillaz - new gold ft. tame impala & bootie b...   
3     gorillaz - on melancholy hill (official video)   
4         gorillaz - clint eastwood (official video)   

                                           

### 3. Filtrado de datos:

In [9]:
'''import numpy as np

# Convertir las columnas 'Licensed' y 'official_video' a booleanas
df['Licensed'] = df['Licensed'].map({'True': True, 'False': False})
df['official_video'] = df['official_video'].map({'True': True, 'False': False})

# Manejo de valores nulos
# Usar la mediana para completar valores nulos en características musicales
musical_columns = ['Danceability', 'Energy', 'Key', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo', 'Duration_ms']
df[musical_columns] = df[musical_columns].fillna(df[musical_columns].median())

# Usar la mediana para completar valores nulos en métricas de YouTube
youtube_columns = ['Views', 'Likes', 'Comments', 'Stream']
df[youtube_columns] = df[youtube_columns].fillna(df[youtube_columns].median())

# Completar URLs faltantes o eliminar filas según sea necesario
df.dropna(subset=['Url_spotify', 'Url_youtube'], inplace=True)

# Crear nuevas columnas para ratio de interacción
df['Likes_to_Views'] = df['Likes'] / df['Views']
df['Comments_to_Views'] = df['Comments'] / df['Views']

# Resumen del DataFrame después de la limpieza
print(df.info())

# Análisis Exploratorio
import matplotlib.pyplot as plt
import seaborn as sns

# Distribución de Views
plt.figure(figsize=(10, 6))
sns.histplot(df['Views'], bins=50, kde=True)
plt.title('Distribución de Views')
plt.xlabel('Views')
plt.ylabel('Frecuencia')
plt.show()

# Correlación entre características musicales y Streams
plt.figure(figsize=(10, 6))
sns.heatmap(df[musical_columns + ['Stream']].corr(), annot=True, cmap='coolwarm')
plt.title('Correlación entre Características Musicales y Streams')
plt.show()  '''


"import numpy as np\n\n# Convertir las columnas 'Licensed' y 'official_video' a booleanas\ndf['Licensed'] = df['Licensed'].map({'True': True, 'False': False})\ndf['official_video'] = df['official_video'].map({'True': True, 'False': False})\n\n# Manejo de valores nulos\n# Usar la mediana para completar valores nulos en características musicales\nmusical_columns = ['Danceability', 'Energy', 'Key', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo', 'Duration_ms']\ndf[musical_columns] = df[musical_columns].fillna(df[musical_columns].median())\n\n# Usar la mediana para completar valores nulos en métricas de YouTube\nyoutube_columns = ['Views', 'Likes', 'Comments', 'Stream']\ndf[youtube_columns] = df[youtube_columns].fillna(df[youtube_columns].median())\n\n# Completar URLs faltantes o eliminar filas según sea necesario\ndf.dropna(subset=['Url_spotify', 'Url_youtube'], inplace=True)\n\n# Crear nuevas columnas para ratio de interacción\ndf['Likes_to_